In [0]:
-- SALES HISTORY --

with sales_history as (
  select a.flightkey, a.dtg, a.unt_net as sales, to_date(a.charge_dt, 'yyyyMMdd') as charge_dt, to_date(b.onsale_dt, 'yyyyMMdd') as first_date, least(to_date(left(a.flightkey, 8), 'yyyyMMdd'), to_date(current_date(), 'yyyyMMdd')) as last_date, date_diff(to_date(left(a.flightkey, 8), 'yyyyMMdd'), b.onsale_dt) as sale_length
  from data_experience_commercial.cbt_1423_rtsuite.master_uat a
  left join data_experience_commercial.cbt_0923_segmentfinder.dimensions_history b 
  on a.flightkey=b.flightkey
  where 1=1
  and a.chargeproduct = 'Ticket'
  and to_date(a.charge_dt, 'yyyyMMdd') <= least(to_date(left(a.flightkey, 8), 'yyyyMMdd'), to_date(current_date(), 'yyyyMMdd'))
  and to_date(a.charge_dt, 'yyyyMMdd') >= to_date(b.onsale_dt, 'yyyyMMdd')
  and date_diff(to_date(a.charge_dt, 'yyyyMMdd'), b.onsale_dt) >= 7
  and to_date(left(a.flightkey, 8), 'yyyyMMdd') >= '2025-04-01'
  and to_date(left(a.flightkey, 8), 'yyyyMMdd') <= '2025-06-30'
),

flightkeydtg_sales as (
  select flightkey, charge_dt, sum(sales) as sales, first(first_date) as first_date, first(last_date) as last_date
  from sales_history
  group by flightkey, charge_dt
),

flightkey_sales as (
  select flightkey, first(first_date) as first_date, first(last_date) as last_date
  from flightkeydtg_sales
  group by flightkey
),

datespine_sales as (
  select flightkey, explode(sequence(first_date, last_date, interval 1 day)) as calender_date
  from flightkey_sales
),

filled_sales as (select a.flightkey, a.calender_date as date, ifnull(b.sales, 0) as sales, sum(ifnull(b.sales, 0)) over (partition by a.flightkey order by a.calender_date asc rows between unbounded preceding and current row) as cumulative_sales
from datespine_sales a
left join flightkeydtg_sales b
on a.flightkey=b.flightkey and a.calender_date=b.charge_dt
where 1=1
order by a.calender_date),

-- PRICE HISTORY --

price_history as (
  select a.segment, a.standardprice, a.currencycode, to_date(a.when, 'yyyyMMdd') as charge_dt, to_date(b.onsale_dt, 'yyyyMMdd') as first_date, least(to_date(left(a.segment, 8), 'yyyyMMdd'), to_date(current_date(), 'yyyyMMdd')) as last_date
  from data_prod.silver_sanezdb.priceinspection a
  left join data_experience_commercial.cbt_0923_segmentfinder.dimensions_history b 
  on a.segment=b.flightkey
  where 1=1
  and to_date(a.when, 'yyyyMMdd') <= least(to_date(left(a.segment, 8), 'yyyyMMdd'), to_date(current_date(), 'yyyyMMdd'))
  and to_date(a.when, 'yyyyMMdd') >= to_date(b.onsale_dt, 'yyyyMMdd')
  and to_date(left(a.segment, 8), 'yyyyMMdd') >= '2025-04-01'
  and to_date(left(a.segment, 8), 'yyyyMMdd') <= '2025-06-30'
),

segmentdtg_price as (
  select segment, charge_dt, avg(standardprice) as price, first(currencycode) as currency, first(first_date) as first_date, first(last_date) as last_date
  from price_history
  group by segment, charge_dt
),

segment_price as (
  select segment, first(first_date) as first_date, first(last_date) as last_date
  from segmentdtg_price
  group by segment
),

datespine_price as (
  select segment, explode(sequence(first_date, last_date, interval 1 day)) as calender_date
  from segment_price
),

body_price as (
  select a.segment, a.calender_date, b.price, b.currency
  from datespine_price a
  left join segmentdtg_price b
  on a.segment=b.segment and a.calender_date=b.charge_dt
),

ffill_price as (
  select segment, calender_date, last_value(price) ignore nulls over (partition by segment order by calender_date rows between unbounded preceding and current row) as price_ffill, last_value(currency) ignore nulls over (partition by segment order by calender_date rows between unbounded preceding and current row) as currency_ffill
  from body_price
),

filled_price as (
  select segment, calender_date as date, coalesce(price_ffill, first_value(price_ffill) ignore nulls over (partition by segment order by calender_date rows between current row and unbounded following)) as price_filled, coalesce(currency_ffill, first_value(currency_ffill) ignore nulls over (partition by segment order by calender_date rows between current row and unbounded following)) as currency_filled
  from ffill_price
),

-- PROFILE HISTORY --

profile_history as (
  select a.flightkey, to_date(a.insertdttm, 'yyyyMMdd') as date, least(to_date(left(a.flightkey, 8), 'yyyyMMdd'), to_date(current_date(), 'yyyyMMdd')) as last_date, to_date(b.onsale_dt, 'yyyyMMdd') as first_date, a.rms_profile, a.rms_profile_tgt
  from data_experience_commercial.cbt_0923_segmentfinder.rms_history a
  left join data_experience_commercial.cbt_0923_segmentfinder.dimensions_history b
  on a.flightkey = b.flightkey
  where 1=1
  and to_date(a.insertdttm, 'yyyyMMdd') < least(to_date(left(a.flightkey, 8), 'yyyyMMdd'), to_date(current_date(), 'yyyyMMdd'))
  and to_date(left(a.flightkey, 8), 'yyyyMMdd') >= '2025-04-01'
  and to_date(left(a.flightkey, 8), 'yyyyMMdd') <= '2025-06-30'
),

flightkey_profiles as (
  select flightkey, first(first_date) as first_date, first(last_date) as last_date
  from profile_history
  group by flightkey
),

datespine_profiles as (
  select flightkey, explode(sequence(first_date, last_date, interval 1 day)) as calender_date
  from flightkey_profiles
),

body_profiles as (
  select a.flightkey, a.calender_date, b.rms_profile, b.rms_profile_tgt
  from datespine_profiles a
  left join profile_history b
  on a.flightkey=b.flightkey and a.calender_date=b.date
),

ffill_profiles as (
  select flightkey, calender_date, last_value(rms_profile) ignore nulls over (partition by flightkey order by calender_date rows between unbounded preceding and current row) as rms_profile_ffill, rms_profile_tgt
  from body_profiles
),

filled_profiles as (
  select flightkey, calender_date as date, coalesce(rms_profile_ffill, first_value(rms_profile_ffill) ignore nulls over (partition by flightkey order by calender_date rows between current row and unbounded following)) as profile_filled, rms_profile_tgt as profile_tgt
  from ffill_profiles
),

-- ROS HISTORY --
 
ros_history as (
  select segment, to_date(datestamp, 'yyyyMMdd') as date, username, breakpointprofile
  from data_prod.silver_sanezdb.rmsbreakpointhistory
  where 1=1
  and username like '%ROS%'
  and breakpointprofile like '%GB%'
  and to_date(left(segment, 8), 'yyyyMMdd') >= '2025-04-01'
  and to_date(left(segment, 8), 'yyyyMMdd') <= '2025-06-30'
),

-- COMBINED HISTORY --


combined_history  as (
  select a.flightkey, to_date(left(a.flightkey, 8), 'yyyyMMdd') as flight_dt, a.date as charge_dt, date_diff(to_date(left(a.flightkey, 8), 'yyyyMMdd'), a.date) as dtg, a.sales, a.cumulative_sales, b.price_filled, b.currency_filled, c.profile_filled, c.profile_tgt, d.username, d.breakpointprofile,
  e.sector, e.route, e.parentregion as region, e.routetype, e.ty_capacity, e.flight_std as flight_time, e.flight_dow
  from filled_sales a
  left join filled_price b
  on a.flightkey = b.segment and a.date = b.date
  left join filled_profiles c
  on a.flightkey = c.flightkey and a.date = c.date
  left join ros_history d
  on a.flightkey = d.segment and a.date = d.date
  left join data_experience_commercial.cbt_0923_segmentfinder.dimensions_history e
  on a.flightkey = e.flightkey
),

lag_cols as (
  select *, lag(sales, 1) over w as sales_lag_1, lag(sales, 2) over w as sales_lag_2, lag(sales, 3) over w as sales_lag_3, lag(sales, 7) over w as lag_sales_7, lag(sales, 14) over w as lag_sales_14, lag(sales, 21) over w as lag_sales_21, lag(sales, 28) over w as lag_sales_28,
  lead(sales, 1) over w as sales_lead_1, lead(sales, 2) over w as sales_lead_2, lead(sales, 3) over w as sales_lead_3,
  lag(price_filled, 1) over w as price_lag_1, lead(price_filled, 1) OVER w AS price_lead_1
  from combined_history
  window w as (partition by flightkey order by charge_dt)
),

treatment_outcome as (
  select *, ln(price_lead_1) - ln(price_lag_1) as treatment_delta, ln(sales_lead_1 + sales_lead_2 + sales_lead_3 + 1) - ln(sales_lag_1 + sales_lag_2 + sales_lag_3 + 1) as outcome_delta, ln(sales_lag_1 + sales_lag_2 + sales_lag_3 + 1) as log_sales_pre, ln(sales_lead_1 + sales_lead_2 + sales_lead_3 + 1) as log_sales_post, ln(price_lead_1) as log_price_post, ln(price_lag_1) as log_price_pre
  from lag_cols where username like '%ROS%'
),

filtered as (
  select * from treatment_outcome
  where (username like '%Increase%' and treatment_delta > 0.05) or (username like '%Decrease%' and treatment_delta < -0.05)
),

profile_extract as (
  select *, 
  case
    when profile_filled like '%:%' then 0
    else cast(split(profile_filled, '_')[0] as int)
  end as x100,
  case
    when profile_filled like '%:%' then cast(split(split(profile_filled, ',')[0], ':')[1] as int)
    else cast(split(profile_filled, '_')[1] as int)
  end as x60,
  case
    when profile_filled like '%:%' then cast(split(split(profile_filled, ',')[1], ':')[1] as int)
    else cast(split(profile_filled, '_')[2] as int)
  end as x20,
  case
    when profile_filled like '%:%' then cast(split(profile_filled, ',')[2] as int)
    else cast(split(profile_filled, '_')[3] as int)
  end as x00
  from filtered where profile_filled like '%:%' or profile_filled like '%_%'
),

markets as (
    select sector, first(sectormarketpair) as sectormarketpair
    from data_experience_commercial.cbt_0899_competitordashboard.competitoralerts_history
    group by sector
),

final_combined as (
  select a.flightkey, a.flight_dt, a.flight_time, a.flight_dow, a.charge_dt, a.dtg, a.sales, a.cumulative_sales, a.price_filled, a.currency_filled, a.sector, b.sectormarketpair as market_sector, a.route, a.region, a.routetype, a.ty_capacity, a.treatment_delta, a.outcome_delta, a.log_sales_pre, a.log_sales_post, a.log_price_pre, a.log_price_post, a.x100, a.x60, a.x20, a.x00,
  coalesce(a.lag_sales_7, 0) as lag_sales_7, coalesce(a.lag_sales_14, 0) as lag_sales_14, coalesce(a.lag_sales_21, 0) as lag_sales_21, coalesce(a.lag_sales_28, 0) as lag_sales_28
  from profile_extract a
  inner join markets b
  on a.sector=b.sector
),

comp as (
  select to_date(observation_date, 'yyyyMMdd') as observation_date, origin, destination, sector, carrier, --b.carriertype
  to_date(outbound_departure_date, 'yyyyMMdd') as flight_dt, cast(concat(date_format(to_date(outbound_departure_date, 'yyyyMMdd'), 'yyyy-MM-dd'), ' ', outbound_departure_time) as timestamp) as comp_flight_time, days_to_go, currency,
  case
    when is_one_way = 1 then price_inc
    else
      case
        when price_outbound is null then price_inc/2
        else price_outbound + (coalesce(tax, 0)/2)
      end
  end as comp_price
  from data_prod.silver_curated_infare.infaredata
  where 1=1
  and to_date(outbound_departure_date, 'yyyyMMdd') >= '2025-04-01'
  and to_date(outbound_departure_date, 'yyyyMMdd') <= '2025-06-30' 
),

comp_curated as (
  select observation_date, sector, carrier, flight_dt, comp_flight_time, first(days_to_go) as dtg, avg(comp_price) as comp_price, first(currency) as currency--, first(carriertype) as carrier_type
  from comp group by observation_date, sector, carrier, flight_dt, comp_flight_time
),

comp_curated_market as (
  select a.*, b.sectormarketpair as market_sector
  from comp_curated a
  inner join markets b
  on a.sector=b.sector
),

optionality as (
  select a.flightkey, a.region, a.routetype, a.flight_dow, a.dtg, b.comp_flight_time, b.comp_price, b.currency, (unix_timestamp(b.comp_flight_time) - unix_timestamp(a.flight_time))/3600 as time_diff, b.comp_price/a.price_filled as price_ratio, cast(substring(b.comp_flight_time, 12, 2) as int) as comp_flight_hod
  from final_combined a 
  left join comp_curated_market b
  on a.market_sector = b.market_sector and a.flight_dt = b.flight_dt and a.dtg=b.dtg
  where a.currency_filled = b.currency
),

final_RPS as (
  select flightkey, max(ty_ticketrev_net) as final_rev
  from data_experience_commercial.cbt_0923_segmentfinder.ticket_history
  where 1=1
  and to_date(left(flightkey, 8), 'yyyyMMdd') >= '2024-04-01'
  and to_date(left(flightkey, 8), 'yyyyMMdd') <= '2024-06-30'
  group by flightkey
),

hourly_aggregates as (
    select a.parentregion, a.routetype, a.flight_dow, a.flight_hod, avg(b.final_rev)/avg(a.ty_capacity) as final_RPS
    from data_experience_commercial.cbt_0923_segmentfinder.dimensions_history a
    left join final_RPS b
    on a.flightkey = b.flightkey
    and a.flight_dt between '2024-04-01' and '2024-06-30'
    group by a.parentregion, a.routetype, a.flight_dow, a.flight_hod
),

scaled_quality_scores as (
    select *, min(final_RPS) over (partition by parentregion, routetype) as min_rps, max(final_RPS) over (partition by parentregion, routetype) as max_rps,
        case
            when max(final_RPS) over (partition by parentregion, routetype) = min(final_RPS) over (partition by parentregion, routetype) 
            then 0.5 -- Default value if all RPS are identical
            else 
                ((final_RPS - min(final_RPS) over (partition by parentregion, routetype)) / (max(final_RPS) over (partition by parentregion, routetype) - min(final_RPS) over (partition by parentregion, routetype))) + 1
        end as time_quality_score
    from 
        hourly_aggregates
),

time_score as (
  select a.*, b.time_quality_score
  from optionality a
  left join scaled_quality_scores b
  on a.region = b.parentregion and a.routetype = b.routetype and a.flight_dow = b.flight_dow and a.comp_flight_hod = b.flight_hod),


optionality_score as (select flightkey, dtg, sum(time_quality_score/((power(price_ratio, 2))*(power((abs(time_diff) + 1), 2)))) as optionality_score
  from time_score
  group by flightkey, dtg),

combined_with_metrics as (
  select a.*, coalesce(b.optionality_score, 0) as optionality_score
  from final_combined a
  left join optionality_score b
  on a.flightkey=b.flightkey and a.dtg=b.dtg),

flight_markets as (
  select a.flightkey, b.sectormarketpair as market_sector
  from data_experience_commercial.cbt_0923_segmentfinder.dimensions_history a
  left join markets b
  on a.sector = b.sector
  where 1=1
  and to_date(left(a.flightkey, 8), 'yyyyMMdd') >= '2025-04-01'
  and to_date(left(a.flightkey, 8), 'yyyyMMdd') <= '2025-06-30'
),

flight_market_prices as (
  select a.segment, to_date(left(a.segment, 8), 'yyyyMMdd') as flight_dt, a.date as charge_dt, a.price_filled, b.market_sector
  from filled_price a
  left join flight_markets b
  on a.segment = b.flightkey
),

neighbours as (
  select a.flightkey, a.dtg, a.flight_time, a.price_filled, a.region, a.routetype, a.flight_dow, b.segment as neighbour_flightkey, b.price_filled as neighbour_price, b.price_filled/a.price_filled as neighbour_price_ratio
  from combined_with_metrics a
  left join flight_market_prices b
  on a.market_sector = b.market_sector and a.flight_dt = b.flight_dt and a.charge_dt = b.charge_dt
  where a.flightkey <> b.segment),

neighbour_time as (
  select a.*, b.flight_std as neighbor_flight_time, (unix_timestamp(a.flight_time) - unix_timestamp(b.flight_std))/3600 as neighbour_time_diff, cast(substring(b.flight_std, 12, 2) as int) as neighbour_flight_hod
  from neighbours a
  left join data_experience_commercial.cbt_0923_segmentfinder.dimensions_history b
  on a.neighbour_flightkey=b.flightkey
),

neighbour_time_score as (
  select a.*, b.time_quality_score as neighbour_time_quality_score
  from neighbour_time a
  left join scaled_quality_scores b
  on a.region = b.parentregion and a.routetype = b.routetype and a.flight_dow = b.flight_dow and a.neighbour_flight_hod = b.flight_hod
),

internal_optionality_score as (
  select flightkey, dtg, sum(neighbour_time_quality_score/((power(neighbour_price_ratio, 2))*(power((abs(neighbour_time_diff) + 1), 2)))) as internal_optionality_score
  from neighbour_time_score
  group by flightkey, dtg
),

total_optionality as (
  select a.*, coalesce(b.internal_optionality_score, 0) as internal_optionality_score, a.optionality_score + coalesce(b.internal_optionality_score, 0) as total_optionality_score, left(sector, 3) as origin_airport, right(sector, 3) as destination_airport
  from combined_with_metrics a
  left join internal_optionality_score b
  on a.flightkey=b.flightkey and a.dtg=b.dtg
),

business_penetration as (
  select first_flightkey, left(first_sector, 3) as origin_airport, right(first_sector, 3) as destination_airport, bl_flag
  from data_experience_commercial.cbt_1470_reservationattributes.augmented
  where to_date(left(first_flightkey, 8), 'yyyyMMdd') between '2024-04-01' and '2024-06-30' 
),

origin_airport_BP as (
    select origin_airport, avg(case when bl_flag = 'Business' then 1.0 else 0.0 end) as origin_bp
    from business_penetration
    group by origin_airport
),

destination_airport_BP as (
    select destination_airport, avg(case when bl_flag = 'Business' then 1.0 else 0.0 end) as destination_bp
    from business_penetration
    group by destination_airport
),

total_BP as (
  select a.*, b.origin_bp, c.destination_bp, b.origin_bp*c.destination_bp as combined_bp
  from total_optionality a
  left join origin_airport_BP b
  on a.origin_airport=b.origin_airport
  left join destination_airport_BP c
  on a.destination_airport=c.destination_airport),

curated_BP as (
  select flightkey, flight_dt, flight_time, charge_dt, dayofweek(flight_dt) as flight_dow, dayofweek(charge_dt) AS charge_dow, dayofmonth(flight_dt) as flight_dom, dayofmonth(charge_dt) as charge_dom, month(flight_dt) as flight_mth, month(charge_dt) as charge_mth, dtg, route, region, routetype,
  ty_capacity, cumulative_sales, round((cumulative_sales/ty_capacity), 2) as Loadfactor, round(treatment_delta, 6) as treatment_delta, round(outcome_delta, 6) as outcome_delta, round(log_sales_pre, 6) as log_sales_pre, x100, x60, x20, x00, lag_sales_7, lag_sales_14, lag_sales_21, lag_sales_28, round(optionality_score, 6) as external_optionality_score, 
  round(internal_optionality_score, 6) as internal_optionality_score, round(total_optionality_score, 8) as total_optionality_score, round(origin_bp, 6) as origin_bp, round(destination_bp, 6) as destination_bp, round(combined_bp, 6) as combined_bp
  from total_BP),

directionality as (
  select route, left(route, 3) as base, right(route, 3) as dest, avg(case when orgcity = left(route, 3) then 1.0 else 0.0 end) as prop_from_base, avg(case when orgcity = right(route, 3) then 1.0 else 0.0 end) as prop_from_dest
  from data_prod.silver_sanezdb.psrhistory
  where 1=1
  and to_date(left(flightsector, 8), 'yyyyMMdd') between '2022-10-01' and '2025-04-01'
  group by route),

route_directionality as (
  select a.*, b.base, b.dest, b.prop_from_base, b.prop_from_dest
  from curated_BP a
  left join directionality b
  on a.route=b.route
),

onsale_dates as (
  select flightkey, first(onsale_dt) as onsale_dt
  from data_experience_commercial.cbt_0923_segmentfinder.dimensions_history
  where flight_dt between '2025-04-01' and '2025-06-30'
  group by flightkey
),

sale_length as (
  select a.*, datediff(a.flight_dt, b.onsale_dt) as sale_length, round((datediff(a.charge_dt, b.onsale_dt)/datediff(a.flight_dt, b.onsale_dt)), 2) as sale_period_progress
  from route_directionality a
  left join onsale_dates b
  on a.flightkey=b.flightkey
),

flight_time(
  select flightkey, first(parentregion) as parentregion, first(routetype) as routetype, first(flight_dow) as flight_dow, first(flight_hod) as flight_hod
  from data_experience_commercial.cbt_0923_segmentfinder.dimensions_history
  where flight_dt between '2025-04-01' and '2025-06-30'
  group by flightkey
),

final_time as (
  select a.flightkey, b.time_quality_score
  from flight_time a
  left join scaled_quality_scores b
  on a.parentregion=b.parentregion and a.routetype=b.routetype and a.flight_dow=b.flight_dow and a.flight_hod=b.flight_hod
),

final_table as (
  select a.*, round(b.time_quality_score, 4) as time_quality_score
  from sale_length a
  left join final_time b
  on a.flightkey=b.flightkey
)

select * from final_table
where 1=1
and dtg <=252
and dtg >=4
and flight_dt >= '2025-04-01'
and flight_dt <= '2025-06-30';



